In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:

# ATR Calculation
def atr(df, period=14):
    df['H-L'] = df['High'] - df['Low']
    df['H-PC'] = abs(df['High'] - df['Close'].shift(1))
    df['L-PC'] = abs(df['Low'] - df['Close'].shift(1))
    df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
    df['ATR'] = df['TR'].rolling(window=period, min_periods=1).mean()
    return df

# Supertrend calculation
def supertrend(df, period=14, multiplier=3):
    df = atr(df, period)
    df['Upper Band'] = ((df['High'] + df['Low']) / 2) + (multiplier * df['ATR'])
    df['Lower Band'] = ((df['High'] + df['Low']) / 2) - (multiplier * df['ATR'])
    df['In Uptrend'] = True

    for current in range(1, len(df.index)):
        previous = current - 1

        if df['Close'][current] > df['Upper Band'][previous]:
            df['In Uptrend'][current] = True
        elif df['Close'][current] < df['Lower Band'][previous]:
            df['In Uptrend'][current] = False
        else:
            df['In Uptrend'][current] = df['In Uptrend'][previous]

            if df['In Uptrend'][current] and df['Lower Band'][current] < df['Lower Band'][previous]:
                df['Lower Band'][current] = df['Lower Band'][previous]

            if not df['In Uptrend'][current] and df['Upper Band'][current] > df['Upper Band'][previous]:
                df['Upper Band'][current] = df['Upper Band'][previous]

    df['Supertrend'] = np.where(df['In Uptrend'], df['Lower Band'], df['Upper Band'])
    return df


# RSI Calculation
def rsi(df, period=2):
    df['delta'] = df['Close'].diff(1)
    df['gain'] = np.where(df['delta'] > 0, df['delta'], 0)
    df['loss'] = np.where(df['delta'] < 0, abs(df['delta']), 0)
    df['avg_gain'] = df['gain'].rolling(window=period).mean()
    df['avg_loss'] = df['loss'].rolling(window=period).mean()
    df['rs'] = df['avg_gain'] / df['avg_loss']
    df['RSI']= 0.00

    for i in range(0, len(df)):
        if i < period:
            df['RSI'][i] = np.nan
        elif i == period:
            df['RSI'][i] = 100 - (100 / (1 + df['rs'][i] ))
        else:
            df['avg_gain'][i] = (df['avg_gain'][i-1]*(period - 1) + df['gain'][i]) / period
            df['avg_loss'][i] = (df['avg_loss'][i-1]*(period - 1) + df['loss'][i]) / period
            df['rs'][i] = df['avg_gain'][i] / df['avg_loss'][i]
            df['RSI'][i] = 100 - (100 / (1 + df['rs'][i] ))
    
    
    return df

# Define the strategy function
def apply_strategy(df, atr_period=14, supertrend_atr_mult = 3, rsi_period = 2, risk_reward_ratio=1.5, atr_multiplier=1):
    supertrend(df, atr_period, supertrend_atr_mult)
    rsi(df, rsi_period)
    # Create columns for signals, stop loss, take profit, entry price, exit price, and PnL
    df['Buy_Signal'] = np.nan
    df['Take_Profit'] = np.nan
    df['Stop_Loss'] = np.nan
    df['Sell_Signal'] = np.nan
    df['Hold_Signal'] = np.nan
    df['Entry_Price'] = np.nan
    df['Exit_Price'] = np.nan
    df['PnL(%)'] = np.nan
    df['Buy_Date'] = np.nan
    df['Sell_Date'] = np.nan
    
    position = 0  # Track whether we're in a position (1 = in position, 0 = out of position)
    entry_price = 0
    stop_loss = 0
    take_profit = 0
    buy_date = None
    sell_date = None
    sell_type = None

    for i in range(atr_period, len(df)):
        # Buy Condition
        if position == 0 and df['Close'][i] > df['Supertrend'][i] and df['RSI'][i] < 12:
            df.loc[i, 'Buy_Signal'] = 1  # Mark buy signal
            entry_price = round(df['Close'][i],2)
            df.loc[i, 'Entry_Price'] = entry_price
            df.loc[i, 'Buy_Date'] = df['Date'][i]
            buy_date = df.Date[i]
            
            # Set Stop Loss and Take Profit
            atr_value = df['ATR'][i]
            stop_loss = round(entry_price - atr_value * atr_multiplier, 2)
            take_profit = round(entry_price + atr_value * atr_multiplier * risk_reward_ratio,2)
            
            df.loc[i, 'Stop_Loss'] = stop_loss
            df.loc[i, 'Take_Profit'] = take_profit
            
            position = 1  # Enter position
        
        # While holding a position
        if position == 1:
            df.loc[i, 'Hold_Signal'] = 1  # Mark holding signal
            
            # Keep Entry_Price, Stop_Loss, Take_Profit consistent until sell
            df.loc[i, 'Entry_Price'] = entry_price
            df.loc[i, 'Stop_Loss'] = stop_loss
            df.loc[i, 'Take_Profit'] = take_profit
            
            # Calculate PnL percentage
            current_price = round(df['Close'][i], 2)
            pnl_percentage = round(((current_price - entry_price) / entry_price) * 100, 2)
            df.loc[i, 'PnL(%)'] = pnl_percentage
            df.loc[i, 'Buy_Date'] = buy_date
        
        # Sell Condition
        if df.loc[i-1, 'Hold_Signal'] == 1:
            if df['High'][i] > take_profit:  # Take profit hit
                df.loc[i, 'Sell_Signal'] = 1
                df.loc[i, 'Exit_Price'] = take_profit
                df.loc[i, 'Sell_Date'] = df['Date'][i]
                pnl_percentage = ((take_profit - entry_price) / entry_price) * 100
                df.loc[i, 'PnL(%)'] = pnl_percentage
                sell_type = 'TP'
                position = 0  # Exit position
            elif df['Low'][i] < stop_loss:  # Stop loss hit
                df.loc[i, 'Sell_Signal'] = 1
                df.loc[i, 'Exit_Price'] = stop_loss
                df.loc[i, 'Sell_Date'] = df['Date'][i]
                pnl_percentage = ((stop_loss - entry_price) / entry_price) * 100
                df.loc[i, 'PnL(%)'] = pnl_percentage
                sell_type = 'SL'
                position = 0  # Exit position
        
        # Present Sell Signal
        if df.loc[i-1, 'Hold_Signal'] == 1 and df.loc[i-1, 'Sell_Signal'] == 1:
            df.loc[i, 'Sell_Signal'] = 1
            if sell_type == 'TP':
                df.loc[i, 'Exit_Price'] = take_profit
            elif sell_type == 'SL':
                df.loc[i, 'Exit_Price'] = stop_loss
            df.loc[i, 'Sell_Date'] = df['Date'][i]
            df.loc[i, 'Buy_Date'] = buy_date
            df.loc[i, 'Entry_Price'] = entry_price
            df.loc[i, 'Stop_Loss'] = stop_loss
            df.loc[i, 'Take_Profit'] = take_profit

            df.loc[i, 'PnL(%)'] = pnl_percentage
            position = 0

        # Reset values after sell
        if position == 0 and df['Sell_Signal'][i] == 1 and df.loc[i-1, 'Hold_Signal'] == 0:
            entry_price = np.nan
            stop_loss = np.nan
            take_profit = np.nan
            buy_date = np.nan
            sell_date = np.nan

    # Determine Recommendation
    df['Recommendation'] = np.where(df['Buy_Signal'] == 1, 'Buy', 
                                     np.where(df['Hold_Signal'] == 1, 'Hold', 
                                              np.where(df['Sell_Signal'] == 1, 'Sell', 'None')))
    
    df['Buy_Date'] = pd.to_datetime(df['Buy_Date'])

    # Return the DataFrame with required columns
    return df[['Buy_Date', 'Entry_Price', 'Stop_Loss', 'Take_Profit', 'Close', 'PnL(%)', 'Recommendation']].rename(columns={'Close': 'Current_Price'})


In [3]:
# Get data and clean it
def get_data(ticker):
    df = pd.read_csv(f'{ticker}_3mo_adjusted.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.drop_duplicates()
    df = df.sort_values(by='Date', ascending=True)
        
    return df

In [5]:
# ead the 'sp500_list.csv' file
sp500_list = pd.read_csv('sp500_list.csv')

# Apply the strategy for each stock in 'sp500_list.csv'
final_results = pd.DataFrame()

for idx, row in sp500_list.head(30).iterrows():
    ticker = row['Ticker']
    industry = row['Industry']

    # Get stock dataframe
    stock_df = get_data(ticker)
    
    # Apply strategy to the DataFrame of the current ticker
    stock_result = apply_strategy(stock_df)
    
    # Add Ticker and Sector columns to the result
    stock_result['Ticker'] = ticker
    stock_result['Industry'] = industry
    
    # Combine all results into a single DataFrame
    final_results = pd.concat([final_results, stock_result.iloc[[-1]]], ignore_index= True )
    #final_results = pd.concat([final_results, stock_result], ignore_index= True )

# Display or save the final result
display(final_results[['Ticker', 'Industry', 'Buy_Date', 'Entry_Price', 'Stop_Loss', 'Take_Profit', 'Current_Price', 'PnL(%)', 'Recommendation']].dropna())


,Ticker,Industry,Buy_Date,Entry_Price,Stop_Loss,Take_Profit,Current_Price,PnL(%),Recommendation
3,ABBV,Biotechnology,2024-12-04,176.46,172.70,182.10,173.369995,-2.130795,Sell
9,A,Life Sciences Tools & Services,2024-12-13,139.00,135.43,144.35,139.000000,0.000000,Buy
13,ALB,Specialty Chemicals,2024-12-13,99.38,95.15,105.73,99.379997,0.000000,Buy
24,AEE,Multi-Utilities,2024-12-12,89.79,88.48,91.76,89.839996,0.000000,Sell
